# 9. Household power consumption

Dans ce notebook nous étudions la pertinence du clustering sur une série temporelle de consommation d'électricité d'un foyer. Le dataset étudié contient la consommation en électricité d'un foyer toutes les minutes de 2006 à 2010.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## 9.1 Chargement du dataset

Chargez le dataset `data/household_power_consumption` puis affichez quelques lignes et caractéristiques.

Note : transformez le type de donnée de l'index (`df.index`) de la DataFrame en une date avec la méthode [`pandas.to_datetime`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html).

In [ ]:
dataset_url = '/data/household_power_consumption.csv'
df = pd.read_csv(dataset_url, index_col=0)
df.index = pd.to_datetime(df.index)
print(df.shape)
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe()

Vérifiez si ce dataset contient des valeurs manquantes. Et si oui, remplacez ces valeurs manquantes avecc la fonction [`fillna`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) avec le paramètre `method='bfill'` pour reprendre la dernière valeur horaire présente avant la valeur manquante (une méthode plus élaboré reposant sur une interpolation serait probablement utile, il faudrait aussi probablement analyser la répartition de ces valeurs manquantes comme par exemple analyser la durée des plages manquantes).

In [ ]:
df[df.Global_active_power.isna()]

In [ ]:
df = df.fillna(method='bfill')

## 9.2 Transformation du dataset

L'objectif que nous nous fixons est de vérifier si nous pouvons mettre en évidence des profils de consommation quotidien.

Commencez par créer une nouvelle dataframe nommée `df_hourly` contenant la somme des consommations par heure avec la méthode [`resample`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html) en précisant le paramètre `H`. Appliquez ensuite la méthode `sum` à la suite du resample pour calculer la consommation horaire. Puis affichez un extrait de cette nouvelle dataframe.

In [ ]:
df_hourly = df.resample('H').sum()

In [ ]:
df_hourly.head()

Transformez la dataframe de consommation horaire pour que son index ne soit plus que la date du jour et qu'elle possède une colonne indiquant l'heure. Vous pouvez vous aider de la documentation sur les [`DatetimeIndex`](https://pandas.pydata.org/docs/reference/api/pandas.DatetimeIndex.html).

In [ ]:
df_hourly['hour'] = df_hourly.index.hour
df_hourly.index = df_hourly.index.date
df_hourly

Enfin, appliquez la méthode `pivot` sur cette dataframe avec comme paramètre `columns='hour'`. Stockez le résultat dans une variable `df_pivot`. A quoi correspond cette nouvelle dataframe ?

In [ ]:
df_pivot = df_hourly.pivot(columns='hour')
df_pivot.head()

Les colonnes de cette dataframe sont un [multi-index](https://pandas.pydata.org/docs/user_guide/advanced.html) qui sera génant pour l'affichage. Transformez les colonnes pour ne conserver que les heures (sans le premier niveau de l'index qui n'indique que `Global_active_power`) en utilisant la méthode [`get_level_values`](https://pandas.pydata.org/docs/reference/api/pandas.Index.get_level_values.html) sur l'attribut `columns` de votre dataframe pivot (n'oubliez pas d'écraser l'atttribut `columns` avec le résultat de cette opération) :

In [ ]:
df_pivot.columns = df_pivot.columns.get_level_values(1)
df_pivot.head()

Est-ce que cette nouvelle dataframe contient des valeurs manquantes ? Si oui, pourquoi et que devriez-vous faire ?

In [ ]:
df_pivot[df_pivot.isna().any(axis='columns')]

Ces valeurs manquantes correspondant au premier jour des mesures, nous pouvons supprimer ce jour :

In [ ]:
df_pivot = df_pivot.dropna()

Utilisez la méthode [`plot`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html) pour afficher la consommation horaire pour tous les jours de notre dataset sur un même graphique. Pour cela, utilisez la version transposée de notre dataframe (attribut `T`, affichez cette transposé pour mieux comprendre). Comment pourriez-vous améliorer le rendu ?

In [ ]:
df_pivot.T.plot(figsize=(13,8), legend=False)

In [ ]:
df_pivot.T.plot(figsize=(13,8), color='blue', legend=False, alpha=0.02)

## 9.3 Clustering

Appliquez une série de K-means pour 2 à 31 clusters. Pour chaque nombre de clusters, stockez l'inertie et calculez (et stockez) le [`silhouette_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html), puis affichez l'elbow curve pour le silouhette score et l'inertie.

Note : il est toujours préférable de normaliser vos données avant d'appliquer un K-means.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

silhouette_scores = []
inertia = []
n_cluster_list = np.arange(2,31).astype(int)

X = df_pivot.values.copy()
    
sc = StandardScaler()
X = sc.fit_transform(X)

for n_cluster in n_cluster_list:
    
    kmeans = KMeans(n_clusters=n_cluster)
    cluster_found = kmeans.fit_predict(X)
    silhouette_scores.append(silhouette_score(X, kmeans.labels_))
    inertia.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(n_cluster_list, silhouette_scores)
plt.title('Silhouette scores per cluster size')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(n_cluster_list, inertia)
plt.title('Inertia per cluster size')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

Choisissez un nombre de clusters adéquat, déterminer le cluster pour chaque jour de notre dataset, puis affichez le nombre de jour dans chaque cluster.

In [ ]:
kmeans = KMeans(n_clusters=4)
clusters = kmeans.fit_predict(X)
pd.Series(clusters).value_counts()

Créez une nouvelle colonne sur notre dataframe pivot contenant le numéro de cluster de chaque jour.

In [ ]:
df_pivot['cluster'] = clusters
df_pivot.head()

Calculez la médiane et la moyenne de chaque consommation horaire pour tous les clusters (les [groupes pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#splitting-an-object-into-groups) peuvent permettre de calculer ces médianes). Nommez ces variables `clusters_median` et `clusters_mean`.

In [ ]:
clusters_median = df_pivot.groupby('cluster').median().values
clusters_mean = df_pivot.groupby('cluster').mean().values
# clusters_mean = sc.inverse_transform(kmeans.cluster_centers_)  # alternative using the scaler inverse transform
clusters_median

Si vous avez bien respecté les noms des différentes variables recommandés jusqu'à présent, la cellule suivante devrait afficher un graphique avec les profils des jours médians (ou moyens) de consommation par cluster et les différents jours par cluster. Nous choisissons une palette de couleur pour nos clusters puis nous affichons chaque jour des clusters avec une forte transparence ainsi que le profil médian pour ces clusters. Pour choisir d'afficher le profil moyen, nous pouvons remplacer `clusters_median` par `clusters_mean`.

Comment interprétez vous ces clusters ?

In [ ]:
fig, ax= plt.subplots(1,1, figsize=(18,10))
colors = plt.cm.jet(np.linspace(0,1,kmeans.n_clusters))
colors = ['red', 'blue', 'purple', 'yellow', 'black', 'pink']
for cluster in range(kmeans.n_clusters):
    df_pivot[df_pivot.cluster == cluster].T.plot(
        ax=ax, legend=False, alpha=0.01, color=colors[cluster], label= f'Cluster {cluster}'
    )
    plt.plot(clusters_median[cluster], color=colors[cluster], alpha=1, ls='--', label="lelel")
ax.set_xticks(np.arange(1,25))
ax.set_ylabel('kilowatts')
ax.set_xlabel('hour')
plt.show()

Nous avons mis en évidence plusieurs profils de consommation quotidien, probablement liés à des jours travaillés, de présence (vacances, jours féries ou week-end) et d'absence (vacances ?).

Pour aller plus loin, il serait utile de vérifier si notre hypothèse sur les profils moyen obtenus correspondent bien à des jours travaillés, de présence ou d'absence en utilisant le calendrier locale du dataset (les données ont été collectées en France).